# 데이터 스트림 압축기 – 임계값 버전 (Adaptive Stream Compressor – Threshold)

**TAG:** Greedy, Sliding Window, Monotonic Queue (Deque)

---

## 📄 문제 설명

정수 배열 `nums`와 정수 `K`, 그리고 **임계값** `T`가 주어집니다. 배열을 **여러 개의 연속 구간**으로 분할하려고 합니다. 각 구간 `[i, j]`(포함)의 **압축 다양성 비용**을

```
var(i, j) = max(nums[i..j]) - min(nums[i..j])
```

이라고 할 때, 각 구간에 고정 오버헤드 `K`가 추가로 든다고 하자. 단, **구간이 유효하려면** `var(i, j) + K ≤ T` 를 만족해야 합니다.

배열 전체를 유효한 구간들로 나누되, **필요한 구간의 최소 개수**를 구하세요. (같은 의미로, 전체 비용이 `segments * K + ∑var` 라면, 모든 유효 구간은 상한 `T` 내이므로 **구간 수를 최소화**하는 것이 목표와 동치입니다.)

> 직관: 한 구간의 다양성(`max-min`)과 오버헤드 `K`의 합이 `T`를 넘지 않는 한, 가능한 한 **길게** 구간을 확장하는 것이 전체 구간 수를 줄이는 최선입니다.

---

## ⛔ 제약 조건

* `1 ≤ N = len(nums) ≤ 10^5`
* `1 ≤ nums[i] ≤ 10^9`
* `0 ≤ K ≤ 10^9`, `0 ≤ T ≤ 10^9`
* 항상 적어도 하나의 해가 존재한다고 가정하지 않습니다. **해가 없으면 `-1`을 반환**합니다.

---

## 🧠 예시

### 예시 1

```
입력: nums = [1, 5, 6, 2, 3], K = 2, T = 7
출력: 1
설명: 전체 하나의 구간 [0..4] 에 대해 var = 6-1 = 5, 5 + K = 7 ≤ T 이므로 1개 구간이면 충분.
```

### 예시 2

```
입력: nums = [4, 3, 1, 10, 5, 8], K = 3, T = 8
가능한 출력(1단계 기준): 2
가능한 분할(2단계 기준의 한 예): [0..2], [3..5]
설명: 각 구간은 var + K ≤ 8을 만족해야 함. 그리디하게 확장하다가 조건을 깨면 직전에서 구간을 끊음.
```

> 인덱스는 **0-기반, 구간은 양끝 포함**(`[L..R]`)입니다.

---

## 💡 권장 접근 (힌트)

* **그리디 + 슬라이딩 윈도우**: 왼쪽 포인터 `L`, 오른쪽 포인터 `R`로 윈도우를 확장합니다.
* 윈도우 내 **최대/최소**를 `deque` 두 개(단조 감소/증가)로 유지하면 `O(1)`로 `max-min`을 구할 수 있습니다.
* 조건 `max - min + K ≤ T`를 만족하는 동안 `R`을 늘리고, 조건을 처음으로 **넘어서는 순간 직전에서 구간을 확정**합니다.
* **불가능 판정**: 단일 원소의 다양성은 `0`이므로, `K > T`이면 어떤 구간도 만들 수 없습니다(즉시 `-1`).

**시간 복잡도:** `O(N)` (각 원소가 덱에 들어갔다 나오는 횟수는 최대 1회)
**공간 복잡도:** `O(N)` (최악의 경우 덱에 단조로 원소가 쌓일 수 있음)

---

# ✅ 단계별 구현 과제

두 단계로 나누어 구현하십시오. **각 단계는 독립적으로 채점**됩니다.

## 1단계 (필수) — 최소 구간 개수만 반환

아래 함수를 구현하세요. **최소 구간 수**만 반환합니다. 해가 없으면 `-1`을 반환합니다.

```python
from typing import List

class Solution:
    def minSegmentsUnderThreshold(self, nums: List[int], K: int, T: int) -> int:
        """배열을 var+K ≤ T 인 연속 구간들로 분할할 때 필요한 최소 구간 수.
        불가능하면 -1을 반환.
        """
        pass
```

### 1단계 채점 포인트

* 그리디 확장 후 **조건 위반 시 직전에서 즉시 자르기**
* `K > T`인 경우 **즉시 -1**
* 시간/공간 복잡도 준수 (`O(N)`/`O(N)`)

### 1단계 예제 입출력(참고)

* `([1,5,6,2,3], K=2, T=7) → 1`
* `([4,3,1,10,5,8], K=3, T=8) → 2`
* `([2,2,2], K=1, T=1) → 1`
* `([2,2,2], K=2, T=1) → -1`

---

## 2단계 (확장) — 최소 구간 개수와 **실제 분할 구간**까지 반환

아래 확장 함수를 구현하세요. 2단계에서는 **최소 구간 수와 그 예시 분할(구간 리스트)**를 함께 반환합니다. 여러 최적 해가 있으면, **좌→우 그리디**로 얻는 분할을 반환하면 충분합니다.

```python
from typing import List, Tuple

class Solution:
    def minSegmentsUnderThresholdWithCuts(self, nums: List[int], K: int, T: int) -> Tuple[int, List[Tuple[int, int]]]:
        """최소 구간 수와, 그에 대응하는 [L, R] (0-기반, 양끝 포함) 분할을 반환.
        불가능하면 (-1, []).
        """
        pass
```

### 2단계 출력 형식

* 반환값: `(segments, cuts)`

  * `segments`: 최소 구간 개수 (정수)
  * `cuts`: `[(L1, R1), (L2, R2), ...]` 형태의 구간 리스트 (각 구간은 **연속**, **겹치지 않음**, **전 구간을 정확히 덮음**)
* 불가능한 경우: `(-1, [])`

### 2단계 채점 포인트

* 1단계 로직을 재사용하되 **구간 경계 기록**
* **정확한 커버리지**: `cuts`가 `[0..n-1]`을 정확히 덮어야 함
* `K > T`인 경우 **(-1, [])**

---

## 🧠 학습 포인트 정리

* **그리디**: 가능한 한 긴 유효 구간을 만들어 전체 구간 수를 최소화
* **슬라이딩 윈도우 + 단조 큐**: 윈도우 내 `max/min`을 `O(1)`에 가깝게 유지
* **불가능 판정**: 단일 원소조차 허용되지 않으면 전체도 불가능

> 고급 연결: 본 변형은 임계값 제약 하에서의 **분할 개수 최소화** 문제입니다. 원문(난이도 높음)에서의 비용 직접 최소화(DP/최적화)와 대비해, 본 문제는 **그리디가 최적**임을 관찰하고 효율적으로 구현하는 데 초점을 둡니다.


In [1]:
# ============================================================
# 문제: 임계값 이하 구간 최소 분할 (Minimum Partition Below Threshold)
# 알고리즘: 그리디 + 슬라이딩 윈도우 + 단조 큐 (Monotonic Deque)
# 시간 복잡도: O(n) - 각 원소가 덱에 한 번씩만 들어갔다 나옴
# 공간 복잡도: O(n) - 덱 크기
# ============================================================
#
# 📌 문제 핵심:
# - 배열을 연속 구간들로 분할
# - 각 구간의 비용: var(i,j) + K ≤ T (var = max - min)
# - 목표: 최소 개수의 구간으로 분할
#
# 📌 핵심 통찰 (Key Insight):
# - 구간 수를 최소화하려면 각 구간을 최대한 길게 만들어야 함
# - → 그리디: 조건을 만족하는 한 계속 확장
#
# 📌 효율적인 max/min 관리:
# - 슬라이딩 윈도우 내 max/min을 O(1)에 구하기 위해
# - 단조 감소 덱(max용)과 단조 증가 덱(min용) 사용
#
# 📌 단조 큐 (Monotonic Deque) 설명:
# - 단조 감소 덱: 앞에서 뒤로 갈수록 값이 감소
#   → 앞(front)에 항상 최대값
# - 단조 증가 덱: 앞에서 뒤로 갈수록 값이 증가
#   → 앞(front)에 항상 최소값
# ============================================================

from collections import deque
from typing import List, Tuple

class Solution:
    # ============================================================
    # 1단계: 최소 구간 수만 반환
    # ============================================================
    def minSegmentsUnderThreshold(self, nums: List[int], K: int, T: int) -> int:
        """
        배열을 var+K ≤ T 인 연속 구간들로 분할할 때 필요한 최소 구간 수.
        불가능하면 -1을 반환.
        
        Args:
            nums: 정수 배열
            K: 고정 오버헤드
            T: 임계값
            
        Returns:
            int: 최소 구간 수, 불가능하면 -1
            
        Examples:
            >>> sol = Solution()
            >>> sol.minSegmentsUnderThreshold([1,5,6,2,3], 2, 7)
            1  # 전체가 하나의 구간 (max-min=5, 5+2=7 ≤ 7)
        """
        
        n = len(nums)
        
        # ============================================================
        # 엣지 케이스 처리
        # ============================================================
        # 빈 배열: 구간이 필요 없음
        if n == 0:
            return 0
        
        # 불가능 판정: K > T이면 단일 원소도 구간을 만들 수 없음
        # 단일 원소의 var = 0이므로, 0 + K > T이면 불가능
        if K > T:
            return -1
        
        # ============================================================
        # 그리디 + 슬라이딩 윈도우 + 단조 큐
        # ============================================================
        
        # 단조 큐 초기화
        # max_deque: 단조 감소 (front가 최대), 인덱스 저장
        # min_deque: 단조 증가 (front가 최소), 인덱스 저장
        max_deque = deque()  # 윈도우 내 최대값 추적
        min_deque = deque()  # 윈도우 내 최소값 추적
        
        segments = 0  # 분할된 구간 수
        left = 0      # 현재 구간의 시작 인덱스
        
        for right in range(n):
            # ============================================================
            # Step 1: 새 원소를 덱에 추가 (단조성 유지)
            # ============================================================
            
            # max_deque 업데이트 (단조 감소 유지)
            # 새 원소보다 작은 값들은 더 이상 최대가 될 수 없으므로 제거
            while max_deque and nums[max_deque[-1]] <= nums[right]:
                max_deque.pop()
            max_deque.append(right)
            
            # min_deque 업데이트 (단조 증가 유지)
            # 새 원소보다 큰 값들은 더 이상 최소가 될 수 없으므로 제거
            while min_deque and nums[min_deque[-1]] >= nums[right]:
                min_deque.pop()
            min_deque.append(right)
            
            # ============================================================
            # Step 2: 현재 윈도우의 var 계산 및 조건 확인
            # ============================================================
            current_max = nums[max_deque[0]]
            current_min = nums[min_deque[0]]
            var = current_max - current_min
            
            # ============================================================
            # Step 3: 조건 위반 시 구간 분할
            # ============================================================
            if var + K > T:
                # 조건 위반! 직전 위치(right-1)까지로 구간 확정
                segments += 1
                
                # 새 구간 시작: left = right (현재 원소부터 새 구간)
                left = right
                
                # 덱 초기화: 새 구간에는 현재 원소만 포함
                max_deque.clear()
                min_deque.clear()
                max_deque.append(right)
                min_deque.append(right)
        
        # 마지막 구간 처리 (반복문에서 처리되지 않은 마지막 구간)
        segments += 1
        
        return segments
    
    
    # ============================================================
    # 2단계: 최소 구간 수 + 실제 분할 구간까지 반환
    # ============================================================
    def minSegmentsUnderThresholdWithCuts(
        self, nums: List[int], K: int, T: int
    ) -> Tuple[int, List[Tuple[int, int]]]:
        """
        최소 구간 수와, 그에 대응하는 [L, R] (0-기반, 양끝 포함) 분할을 반환.
        불가능하면 (-1, []).
        
        Args:
            nums: 정수 배열
            K: 고정 오버헤드
            T: 임계값
            
        Returns:
            Tuple[int, List[Tuple[int, int]]]: (구간 수, [(L1,R1), (L2,R2), ...])
            
        Examples:
            >>> sol = Solution()
            >>> sol.minSegmentsUnderThresholdWithCuts([1,5,6,2,3], 2, 7)
            (1, [(0, 4)])
        """
        
        n = len(nums)
        
        # 엣지 케이스
        if n == 0:
            return (0, [])
        
        if K > T:
            return (-1, [])
        
        # 단조 큐 초기화
        max_deque = deque()
        min_deque = deque()
        
        cuts = []     # 분할 구간 리스트
        left = 0      # 현재 구간의 시작 인덱스
        
        for right in range(n):
            # Step 1: 덱 업데이트
            while max_deque and nums[max_deque[-1]] <= nums[right]:
                max_deque.pop()
            max_deque.append(right)
            
            while min_deque and nums[min_deque[-1]] >= nums[right]:
                min_deque.pop()
            min_deque.append(right)
            
            # Step 2: var 계산
            current_max = nums[max_deque[0]]
            current_min = nums[min_deque[0]]
            var = current_max - current_min
            
            # Step 3: 조건 위반 시 구간 분할
            if var + K > T:
                # 구간 [left, right-1] 확정
                cuts.append((left, right - 1))
                
                # 새 구간 시작
                left = right
                
                # 덱 초기화
                max_deque.clear()
                min_deque.clear()
                max_deque.append(right)
                min_deque.append(right)
        
        # 마지막 구간 추가
        cuts.append((left, n - 1))
        
        return (len(cuts), cuts)


# ============================================================
# 알고리즘 시각적 설명
# ============================================================
def visualize_algorithm(nums: List[int], K: int, T: int):
    """
    알고리즘의 동작 과정을 시각적으로 보여줍니다.
    """
    print(f"\n{'='*70}")
    print(f"입력: nums={nums}, K={K}, T={T}")
    print(f"조건: max - min + K ≤ T  →  max - min ≤ {T - K}")
    print(f"{'='*70}")
    
    n = len(nums)
    
    if n == 0:
        print("빈 배열 → (0, [])")
        return
    
    if K > T:
        print(f"K({K}) > T({T}) → 불가능! (-1, [])")
        return
    
    max_deque = deque()
    min_deque = deque()
    cuts = []
    left = 0
    
    print(f"\n[그리디 확장 과정]")
    print("-" * 70)
    
    for right in range(n):
        # 덱 업데이트
        while max_deque and nums[max_deque[-1]] <= nums[right]:
            max_deque.pop()
        max_deque.append(right)
        
        while min_deque and nums[min_deque[-1]] >= nums[right]:
            min_deque.pop()
        min_deque.append(right)
        
        current_max = nums[max_deque[0]]
        current_min = nums[min_deque[0]]
        var = current_max - current_min
        cost = var + K
        
        window = nums[left:right+1]
        status = "✅ 유효" if cost <= T else "❌ 위반"
        
        print(f"right={right}: 윈도우 [{left}..{right}] = {window}")
        print(f"         max={current_max}, min={current_min}, var={var}, cost={cost} {status}")
        
        if var + K > T:
            cuts.append((left, right - 1))
            print(f"         → 구간 확정: [{left}, {right-1}]")
            print(f"         → 새 구간 시작: left={right}")
            left = right
            max_deque.clear()
            min_deque.clear()
            max_deque.append(right)
            min_deque.append(right)
        print()
    
    cuts.append((left, n - 1))
    print(f"마지막 구간 추가: [{left}, {n-1}]")
    
    print(f"\n{'='*70}")
    print(f"결과: {len(cuts)}개 구간")
    print(f"분할: {cuts}")
    print(f"{'='*70}")
    
    # 검증
    print(f"\n[검증]")
    for i, (l, r) in enumerate(cuts):
        segment = nums[l:r+1]
        seg_max = max(segment)
        seg_min = min(segment)
        seg_var = seg_max - seg_min
        seg_cost = seg_var + K
        valid = "✅" if seg_cost <= T else "❌"
        print(f"  구간 {i+1}: [{l},{r}] = {segment}, max={seg_max}, min={seg_min}, cost={seg_cost} ≤ {T}? {valid}")


# ============================================================
# 단조 큐 (Monotonic Deque) 개념 설명
# ============================================================
def explain_monotonic_deque():
    """
    단조 큐의 동작 원리를 설명합니다.
    """
    print("""
╔══════════════════════════════════════════════════════════════════════╗
║                    단조 큐 (Monotonic Deque) 개념                      ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                        ║
║  📌 목적: 슬라이딩 윈도우 내 최대/최소값을 O(1)에 조회                    ║
║                                                                        ║
║  📌 단조 감소 덱 (Max 추적용):                                          ║
║     - 앞(front)에서 뒤(back)로 갈수록 값이 감소                          ║
║     - front에 항상 현재 윈도우의 최대값                                  ║
║                                                                        ║
║     예: [5, 3, 1] → front=5 (최대)                                     ║
║                                                                        ║
║     새 원소 4 추가 시:                                                  ║
║     - 4보다 작은 값들(3, 1) 제거                                        ║
║     - 결과: [5, 4]                                                     ║
║                                                                        ║
║  📌 단조 증가 덱 (Min 추적용):                                          ║
║     - 앞(front)에서 뒤(back)로 갈수록 값이 증가                          ║
║     - front에 항상 현재 윈도우의 최소값                                  ║
║                                                                        ║
║     예: [1, 3, 5] → front=1 (최소)                                     ║
║                                                                        ║
║  📌 시간 복잡도:                                                       ║
║     - 각 원소는 덱에 한 번 들어가고 한 번 나옴                           ║
║     - 전체 O(n), 각 조회 O(1)                                          ║
║                                                                        ║
╚══════════════════════════════════════════════════════════════════════╝
""")


# ============================================================
# 테스트 함수
# ============================================================
def run_tests():
    sol = Solution()
    
    print("=" * 70)
    print("자동 채점 결과")
    print("=" * 70)
    
    # 1단계 테스트
    test_cases_1 = [
        ([1, 5, 6, 2, 3], 2, 7, 1),
        ([4, 3, 1, 10, 5, 8], 3, 8, 2),
        ([2, 2, 2], 1, 1, 1),
        ([2, 2, 2], 2, 1, -1),
        ([1], 0, 0, 1),
        ([], 1, 5, 0),
        ([1, 2, 3, 4, 5], 0, 1, 4),
        ([5, 5, 5, 5], 0, 0, 1),
    ]
    
    print("\n[1단계: 최소 구간 수]")
    score1 = 0
    for i, (nums, K, T, expected) in enumerate(test_cases_1, 1):
        result = sol.minSegmentsUnderThreshold(nums, K, T)
        correct = result == expected
        print(f"Test {i}: nums={nums}, K={K}, T={T}")
        print(f"         결과={result}, 예상={expected} → {'✅' if correct else '❌'}")
        if correct:
            score1 += 1
    print(f"\n1단계 점수: {score1}/{len(test_cases_1)}")
    
    # 2단계 테스트
    print("\n[2단계: 최소 구간 수 + 분할]")
    
    test_cases_2 = [
        ([1, 5, 6, 2, 3], 2, 7),
        ([4, 3, 1, 10, 5, 8], 3, 8),
        ([2, 2, 2], 2, 1),
        ([1], 0, 0),
        ([], 1, 5),
    ]
    
    for nums, K, T in test_cases_2:
        segs, cuts = sol.minSegmentsUnderThresholdWithCuts(nums, K, T)
        print(f"nums={nums}, K={K}, T={T}")
        print(f"  → {segs}개 구간: {cuts}")
        
        # 검증
        if segs > 0 and cuts:
            valid = True
            for l, r in cuts:
                if l > r or (cuts and l != 0 if cuts.index((l,r)) == 0 else False):
                    valid = False
                segment = nums[l:r+1]
                if max(segment) - min(segment) + K > T:
                    valid = False
            print(f"  검증: {'✅ 유효' if valid else '❌ 무효'}")
        print()


# 실행
explain_monotonic_deque()
run_tests()

# 시각화 예시
visualize_algorithm([1, 5, 6, 2, 3], 2, 7)
visualize_algorithm([4, 3, 1, 10, 5, 8], 3, 8)



╔══════════════════════════════════════════════════════════════════════╗
║                    단조 큐 (Monotonic Deque) 개념                      ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                        ║
║  📌 목적: 슬라이딩 윈도우 내 최대/최소값을 O(1)에 조회                    ║
║                                                                        ║
║  📌 단조 감소 덱 (Max 추적용):                                          ║
║     - 앞(front)에서 뒤(back)로 갈수록 값이 감소                          ║
║     - front에 항상 현재 윈도우의 최대값                                  ║
║                                                                        ║
║     예: [5, 3, 1] → front=5 (최대)                                     ║
║                                                                        ║
║     새 원소 4 추가 시:                                                  ║
║     - 4보다 작은 값들(3, 1) 제거                                        ║
║     - 결과: [5, 4]